# Quantum Phase Estimation on the Hubbard molecule

We would like to study the "Hubbard dimer" molecule, whose Hamiltonian reads:

$$H=-t\sum_{\sigma=\uparrow,\downarrow}\left(c_{1\sigma}^{\dagger}c_{2\sigma}+c_{2\sigma}^{\dagger}c_{1\sigma}\right)-\mu\sum_{i=1,2}\sum_{\sigma=\uparrow,\downarrow}n_{i\sigma}+U\sum_{i=1,2}n_{i\uparrow}n_{i\downarrow}$$

with $n_{i\sigma} = c^\dagger_{i\sigma} c_{i\sigma}$. We will choose $\mu=U/2$ to keep the number of electrons at one per site.

## Defining the Hubbard dimer Hamiltonian

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import itertools
from qat.dqs.hamiltonians import make_hubbard_model

U = 1.0
t = 0.2
t_mat = - t * np.array([[0., 1.],
                        [1., 0.]])
hamilt = make_hubbard_model(t_mat, U, mu=U/2)

eigvals = np.linalg.eigvalsh(hamilt.get_matrix())
E0 = min(eigvals)
print("Exact energy = ", E0)

## Performing phase estimation

In [ ]:
from qat.qpus import LinAlg
from qat.dqs.phase_estimation import perform_phase_estimation
qpu = LinAlg()
nqbits_adiab = 6
nqbits_phase = 6
target_energy = E0 + 0.05

print("Guess energy =", target_energy)

for n_adiab_steps, n_trotter_steps in itertools.product([2, 4, 6], [4]):
    energy, prob, eigvec = perform_phase_estimation(hamilt, nqbits_adiab, nqbits_phase,
                                                    n_adiab_steps, n_trotter_steps,
                                                    target_energy, 0.2, 0.2,
                                                    qpu=qpu,
                                                    n_shots=np.inf, verbose=True)
    print("-> n_adiab_steps = %s, n_trotter = %s ==> E = %s"%(n_adiab_steps,n_trotter_steps, energy))

Note that one can of course replace an ideal QPU with a noisy one.

The circuit used for phase estimation is accessible via the function ``make_phase_estimation_circuit``:

In [ ]:
from qat.dqs.phase_estimation import make_phase_estimation_circuit

n_adiab_steps = 2
n_trotter_steps = 1
circ = make_phase_estimation_circuit(hamilt,
                                     nqbits_adiab,
                                     nqbits_phase,
                                     n_adiab_steps,
                                     n_trotter_steps,
                                     target_energy, 0.2, 0.2)

print("Depth of PEA circuit =", len(circ))

## Trotterisation

The ``qat.dqs.trotterisation`` module also provides a trotterization routine (in Jordan-Wigner representation) that returns a ``QRoutine`` approximating the unitary evolution $\exp(-i H t)$. In the cell below, $H$ is ``hamilt``, and $t$ is ``final_time``.

In [ ]:
from qat.dqs.trotterisation import make_trotterisation_routine
from qat.lang.AQASM import Program

n_trotter_steps = 1
final_time = 1

prog = Program()
reg = prog.qalloc(hamilt.hpq.shape[0])
prog.apply(make_trotterisation_routine(hamilt, n_trotter_steps, final_time), reg)

circ = prog.to_circ()

%qatdisplay circ